In [1]:
import subprocess
def ExecuteCmd(command_str, output_file='output_cmd/output.txt'):
  
    with open(output_file, 'w') as f:
        result = subprocess.run(
            command_str,
            shell=True,
            stderr=f, 
            stdout=f,
            text=True,
            executable='/bin/bash'
        )
    return f"Output saved to {output_file}"

In [2]:
from dotenv import load_dotenv
import os
from google import genai

load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

client = genai.Client()

In [3]:
def prompt_tool1(i, scenario, tools, files, questions):
    return f"""
You are tasked with solving a Blue Team Labs Online (BTLO)-style cybersecurity investigation challenge using Ubuntu terminal commands.

Scenario:
{scenario}

Available Tools:
{', '.join(tools)}

Target File(s):
{', '.join(files)}

Question:
{questions[i]}

Instructions:
Based on the question above, generate the **first Ubuntu terminal command** you would execute to begin solving the question.

Respond in this format only:
<your Ubuntu terminal command>

Important:
- Do not answer the question.
- Do not include any explanation.
- Only provide one command.
- Be precise and relevant to the question.
"""


In [4]:
def prompt_tool2(i, scenario, tools, files, questions):
    return f"""
You are assisting in solving a Blue Team Labs Online (BTLO)-style cybersecurity investigation challenge using Ubuntu terminal commands.

Scenario:
{scenario}

Available Tools:
{', '.join(tools)}

Target File(s):
{', '.join(files)}

Question:
{questions[i]}


Instructions:
Based strictly on the previous commands and their outputs and the question:

1. If the answer is **clearly** present in the last output, respond with:
   final answer: <your answer>

2. If the answer is **not yet clear**, and you need to explore further, respond with:
   <your next Ubuntu terminal command>

Important:
- Do **not** include explanations or justifications.
- Only give **one** response per turn, using the correct format.
- Decide carefully whether the answer is present or more steps are needed.
"""


In [5]:
import json

# Path to your JSON file
file_path = "Datasets/file_analysis/fa1.json"

# Load the JSON content
with open(file_path, "r") as f:
    data = json.load(f)

# Extract parts from the JSON
scenario = data["scenario"]
tools = data["tools"]
questions = data["questions"]
file=data["files"]



In [7]:
#custom prompt for this tool: RETURNS ONLY THE COMMAND
def Tool1(history):
    response = client.models.generate_content(
        model="gemini-2.5-pro",
        contents="\n".join([str(h) for h in history])
    )
    #history.append({"role": "model", "text": [f"Command generated by the model:\n{response.text}"]})
    
    return response.text

#not the same prompt as tool1 but take the command provided by tool1
def tool2(history, command):
    with open('output_cmd/output.txt', 'r') as f:
        output = f.read()
    history.append({"previous command": [command], "command output": [output]})

    response = client.models.generate_content(
        model="gemini-2.5-pro",
        contents="\n".join([str(h) for h in history])
    )
    return response.text


In [11]:
i = -1
nb_questions = len(questions)
history = []
final_answers= ["Answer not found"] * nb_questions  # Initialize with default answer
previous_input = None
output1 = None
# Main control loop
while i < nb_questions:
    user_input = input("\nChoose:\n1. Next Question\n2. Analyze/Continue\n3. Exit\n>>> ")
    if previous_input is None and user_input not in ["1", "3"]:
        print("❌ You must start with option 1.")
        continue

    
    if user_input == previous_input and user_input == "1":
        print("❌ You already chose 1. You must alternate between 1 and 2.")
        continue

    previous_input = user_input  # Store the last input for validation

    if user_input == "1":
        if i == nb_questions - 1:
            print("✅ Task finished.")
            break
        i += 1
        prompt = prompt_tool1(i, scenario, tools, file, questions)    #first prompt for tool1
        history = [{"role": "system", "text": [prompt]}]  # start fresh for new question
        output1 = Tool1(history)
        print(f"Command generated for question {i+1} by tool 1: {output1}")
        ExecuteCmd(output1)  # Execute the command generated by Tool1
        #we should modify the history here for the tool2: new history becomes empty -> receives the new prompt -> receives command
        prompt = prompt_tool2(i, scenario, tools, file, questions)    #first prompt for tool2
        history=[{"role": "system", "text": [prompt]}]  # start fresh for new question

    elif user_input == "2":
        output1 = tool2(history, output1)
        print(history)
        if "final answer:" in output1.lower():
            final_answers[i] = output1.lower().split("final answer:")[-1].strip()
            print(f"Answer to the question {i}: {final_answers[i]}, click 1 to move to next question")
            
        else:
            print(f"New command: {output1}, press 2 to execute it ")
            ExecuteCmd(output1)
        
        #if final result append to list else exucute cmd
    elif user_input == "3":
        print("⛔ Stopped by user.")
        break

    else:
        print("⚠️ Invalid input. Please enter 1, 2, or 3.")


Command generated for question 0 by tool 1: grep "From:" Datasets/file_analysis/Your_Account_has_been_locked.eml
[{'role': 'system', 'text': ['\nYou are assisting in solving a Blue Team Labs Online (BTLO)-style cybersecurity investigation challenge using Ubuntu terminal commands.\n\nScenario:\nYou are given a phishing email file in .eml format.\n\nAvailable Tools:\ngrep, awk, sed, ls, base64, cut, sort, wc, head, tail, find, strings, file, stat\n\nTarget File(s):\nDatasets/file_analysis/Your_Account_has_been_locked.eml\n\nQuestion:\nWhat is the sending email address?\n\n\nInstructions:\nBased strictly on the previous commands and their outputs and the question:\n\n1. If the answer is **clearly** present in the last output, respond with:\n   final answer: <your answer>\n\n2. If the answer is **not yet clear**, and you need to explore further, respond with:\n   <your next Ubuntu terminal command>\n\nImportant:\n- Do **not** include explanations or justifications.\n- Only give **one** res

In [13]:
print(final_answers)

['amazon@zyevantoby.cn', 'saintington73', 'your account has been locked', 'amazon', 'wed, 14 jul 2021 01:40:32 +0900', 'Answer not found', 'base64', 'Answer not found', 'Answer not found']
